In [1]:
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr
from mpl_toolkits.basemap import Basemap
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
plt.rcParams['figure.figsize'] = (12,8)
import netCDF4
import time
from progressbar import ProgressBar

In [145]:
flist = []
bdir = '/home/bruno/Documents/AnacondaFiles/ECMWF/'
for yy in range (1997,1998):
    for mm in range(1,13):
        mmfl = sorted(glob(bdir + '/' + str(yy) + '/' + str(mm) + '/' + '*.nc'))
        for fp in mmfl:
            flist.append(fp)
print (len(flist), 'arquivos')

26 arquivos


In [129]:
d = []
MATRIZ = np.shape(5)

pbar = ProgressBar()

for i,j in zip(flist, pbar(range(len(flist)))):
    ds = netCDF4.Dataset(i) # abrindo cada dataset
    pr = ds.variables['pr'] # Acessando a variável 'pr'
    #print('OLHAAAAAA: ', pr)
    #pr_n = pr.sel(longitude = slice(-50, -30), latitude = slice(-20, 10)) # slice em 'lat' e 'lon'
    d.append(np.array(pr)) # adicionando cada 'array pr_n' a list
    MTRAIZ = np.stack(d)
    # concatenando todos os arrays contidos em 'd' ao longo de uma nova dimensão
    pbar.update(j+1)

array = [forecast][time][member][latitude][longitude]

In [ ]:
import xarray as xr
import os

prec = []

for fp in flist:
    if os.path.exists(fp):
        hind = os.path.basename(fp)[23:27]
        yy1 = os.path.basename(fp)[28:32]
        mm1 = os.path.basename(fp)[32:34]
        dd1 = os.path.basename(fp)[34:36]
        yy2 = os.path.basename(fp)[37:41]
        mm2 = os.path.basename(fp)[41:43]
        dd2 = os.path.basename(fp)[43:45]
        id0 = os.path.basename(fp)[46:48]
        d = xr.open_dataset(fp)
        s = d['pr']
        p = s.sel(longitude=slice(-50, -30), latitude=slice(-20, 10))
        #l = [hind, yy1, mm1, dd1, yy2, mm2, dd2, id0, p]
        #print(hind, yy1, mm1, dd1, yy2, mm2, dd2, id0, p)
        prec.append(p)

In [ ]:
def plot_graphic(data, information, step_parallels, step_meridians, resolution_graphic, var):
    '''
    Função para plotagem de gráficos.\n
    Especificações dos parâmetros:\n
        data: Deve receber os dados que serão plotados, como por exemplo: 
            prec[0,[0,3,4]].mean(['member']) => Aqui iremos obter a Precipitação Média\n\t\tentre os Membros 0,3 e 4.\n
        information: Deve receber as informações a serem mostradas no título do gráfico.\n
        step_parallels: A forma como será feito o espaçamento entre as linhas paralelas.\n        
        step_meridians: A forma como será feito o espaçamento entre as linhas meridianas.\n
        var: Booleano que decidirá se o gráfico será salvo.
    '''
    
    m = Basemap(resolution = resolution_graphic, 
            llcrnrlon=longitude.min(),
            urcrnrlon=longitude.max(),
            llcrnrlat=latitude.min(),
            urcrnrlat=latitude.max())
    x, y = m(*np.meshgrid(longitude,latitude))
    m.pcolormesh(x,y,data,shading='flat',cmap=plt.cm.jet)
    m.colorbar(location='right', pad= '7%')
    m.drawcoastlines(linewidth = 1.7)
    m.drawstates(linewidth = 1.)
    m.drawparallels(np.arange(y.min(),y.max(),step_parallels),labels=[1,0,0,0])
    m.drawmeridians(np.arange(x.min(),x.max(),step_meridians),labels=[0,0,0,1])
    plt.xlabel('Longitude', labelpad = 30)
    plt.ylabel('Latitude', labelpad = 35)
    plt.title(information.upper(), pad = 15, linespacing = 1.3)
    plt.rcParams['figure.figsize'] = (10,12)
    
    
    if var == True:
        number_figure = input('Digite o número da figura:\n')
        plt.savefig('figure{}.png'.format(number_figure))